Código em jupyter para poder facilitar a implementação e atualização do meu projeto pessoal de atualização automática de planilha de controle financeiro. Especificamente a planilha de cartões. Dessa vez tentar implementar a automação também do cartão de crédito da caixa.

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd 
from time import sleep
from pprint import pprint
import credenciais
from datetime import datetime, date
from google.oauth2 import service_account
from googleapiclient.discovery import build



In [2]:
options = webdriver.ChromeOptions()
options.add_argument('--log-level=3')
url = 'https://www2.bancobrasil.com.br/aapf/login.html?1638208647003#/acesso-aapf-agencia-conta-1'

chrome = webdriver.Chrome(
    options=options,
    service=ChromeService(ChromeDriverManager().install())
    )


In [3]:
chrome.get(url)
agencia = credenciais.agencia_bb
conta = credenciais.conta_bb
senha = credenciais.senha_bb

In [4]:
# Preencher o campo agência
chrome.find_element(By.ID,"dependenciaOrigem").send_keys(agencia)
# Preencher o campo conta e apertar enter
chrome.find_element(
    By.ID,"numeroContratoOrigem"
    ).send_keys(conta, Keys.ENTER)

In [5]:
# Preencher o campo senha e apertar Enter
chrome.find_element(By.ID,"senhaConta").send_keys(senha, Keys.ENTER)

In [7]:
# Digita Cartões no campo de busca e aperta Enter
chrome.find_element(By.ID,"acheFacil").send_keys("cartões", Keys.ENTER)
sleep(4)
# Escolhe a opção de extrato e entra
act = ActionChains(chrome)
act.click(chrome.find_element(
    By.XPATH,"//a[contains(text(),'Cartões - Fatura - Extrato')]"
    )).perform()

In [8]:
# Seleciona o Cartão Elo Nanquim
chrome.find_element(By.XPATH,'//*[@id="carousel1"]/div/div/img[2]').click()

In [20]:
# Extrair a data de fechamento da próxima fatura
chrome.find_element(By.XPATH,'//*[@id="faturasAtual"]/li[11]/a').click()
sleep(2)
elemento = chrome.find_element(
    By.XPATH,'//*[@id="fatura2"]/table/tbody/tr[4]/td/div/ul/li/span'
    )
html_content = elemento.get_attribute('outerHTML')
soup = BeautifulSoup(html_content, 'html.parser')
fechamento_fatura_elo = soup.get_text('\n')
print(fechamento_fatura_elo)

23/03/2023


In [21]:
# Clica em fatura atual para ver os histórico de compras recente
chrome.find_element(By.XPATH,"//*[@id='faturasAtual']/li[12]").click()

In [22]:
# Rola a página para baixo
chrome.execute_script("window.scrollTo(0,document.body.scrollHeight)")

In [23]:
# Captura a tabela de informações presente na página
elo_nanquim = []
try:
    elemento = chrome.find_element(
        By.XPATH,"//*[@id='fatura2']/div[9]/table")
    html_content = elemento.get_attribute('outerHTML')
    # Método organiza as informações em html
    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.find(name='table')
    # Método que extrai as informações do html e coloca num data frame
    extrato_full = pd.read_html(str(table))[0]
    extrato = extrato_full[[0, 1, 2]]
    extrato.columns = ['Data', 'Descrição', 'Valor']
    extrato = extrato.drop([0, 1], axis=0)
    # Extrai os dados do data frame e manda para um dicionário
    historico_compras = {}
    historico_compras['extrato'] = extrato.to_dict('records')

    lista = []
    elo_nanquim = []
    for linha in historico_compras['extrato']:

        if 'PARC=' in linha['Descrição']:
            xparcelas = int(linha['Descrição'][7])
            lista.append(linha['Data'])
            lista.append(linha['Descrição'][8:-1])
            lista.append(int(linha['Valor'])/100)
            lista.append(xparcelas)
            elo_nanquim.append(lista)
            lista = []
        else:
            lista.append(linha['Data'])
            lista.append(linha['Descrição'])
            lista.append(int(linha['Valor'])/100)
            lista.append(0)
            elo_nanquim.append(lista)
            lista = []
except Exception as err:
    print(err)
pprint(elo_nanquim)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='fatura2']/div[9]/table"}
  (Session info: chrome=109.0.5414.120)
Stacktrace:
Backtrace:
	(No symbol) [0x004D6643]
	(No symbol) [0x0046BE21]
	(No symbol) [0x0036DA9D]
	(No symbol) [0x003A1342]
	(No symbol) [0x003A147B]
	(No symbol) [0x003D8DC2]
	(No symbol) [0x003BFDC4]
	(No symbol) [0x003D6B09]
	(No symbol) [0x003BFB76]
	(No symbol) [0x003949C1]
	(No symbol) [0x00395E5D]
	GetHandleVerifier [0x0074A142+2497106]
	GetHandleVerifier [0x007785D3+2686691]
	GetHandleVerifier [0x0077BB9C+2700460]
	GetHandleVerifier [0x00583B10+635936]
	(No symbol) [0x00474A1F]
	(No symbol) [0x0047A418]
	(No symbol) [0x0047A505]
	(No symbol) [0x0048508B]
	BaseThreadInitThunk [0x757F7D69+25]
	RtlInitializeExceptionChain [0x773BBB9B+107]
	RtlClearBits [0x773BBB1F+191]

[]


In [24]:
# Selecionar o cartão Smiles Visa Infinite
chrome.find_element(By.XPATH,'//*[@id="carousel1"]/div/div/img[3]').click()

In [26]:
# Clica em fatura atual para ver os histórico de compras recente
chrome.find_element(By.CLASS_NAME,"inactive").click()

In [27]:
# Rolar a página para baixo para ter acesso aos gastos recentes
chrome.execute_script("window.scrollTo(0,document.body.scrollHeight)")

In [39]:
# Extrair tabela com os gastos recentes
visa = []
try:
    elemento = chrome.find_element(
        By.XPATH,"//*[@id='fatura2']/div[9]//table")
    html_content = elemento.get_attribute('outerHTML')

    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.find(name='table')

    extrato_full = pd.read_html(str(table))[0]
    extrato = extrato_full[[0, 1, 2]]
    extrato.columns = ['Data', 'Descrição', 'Valor']
    extrato = extrato.drop([0, 1], axis=0)

    historico_compras = {}
    historico_compras['extrato'] = extrato.to_dict('records')

    lista = []
    visa = []
    for linha in historico_compras['extrato']:
        valor = linha['Valor']
        # Condição para quando a compra for maior que um mil reais
        if '.' in valor and ',' in valor:
            valor_mil = valor.replace('.', '')
            valor_mil = float(valor_mil.replace(',','.'))
            lista.append(linha['Data'])
            lista.append(linha['Descrição'])
            lista.append(valor_mil)
            lista.append(0)
            visa.append(lista)
            lista = []
        # Condição caso a compra seja parcelada
        elif 'PARC=' in linha['Descrição']:
            xparcelas = int(linha['Descrição'][7])
            lista.append(linha['Data'])
            lista.append(linha['Descrição'][8:-1])
            lista.append(int(linha['Valor'])/100)
            lista.append(xparcelas)
            visa.append(lista)
            lista = []
        else:
            lista.append(linha['Data'])
            lista.append(linha['Descrição'])
            lista.append(int(linha['Valor'])/100)
            lista.append(0)
            visa.append(lista)
            lista = []
except Exception as err:
    print(err)
pprint(visa)
# chrome.close()

[['10/03', 'ROSANGELA ALMEIDA', 11.0, 0],
 ['10/03', 'CENTRO EDUCACIONAL ADV', 1126.2, 0],
 ['10/03', 'RESTAURANTE DA AN', 30.8, 0],
 ['10/03', 'AUTO POSTO BRAGA LTDA', 40.0, 0],
 ['11/03', 'DROGARIA SAMPAIO', 33.5, 0]]


In [40]:
# Concatenar as tabelas de gastos dos dois cartões
lista_definitiva = visa + elo_nanquim
pprint(lista_definitiva)

[['10/03', 'ROSANGELA ALMEIDA', 11.0, 0],
 ['10/03', 'CENTRO EDUCACIONAL ADV', 1126.2, 0],
 ['10/03', 'RESTAURANTE DA AN', 30.8, 0],
 ['10/03', 'AUTO POSTO BRAGA LTDA', 40.0, 0],
 ['11/03', 'DROGARIA SAMPAIO', 33.5, 0]]


In [41]:
fechamento_fatura = fechamento_fatura_elo
dicionario_meses = {1: "janeiro", 2: "fevereiro", 3: "março",
                    4: "abril", 5: "maio", 6: "junho",
                    7: "julho", 8: "agosto", 9: "setembro",
                    10: "outubro", 11: "novembro", 12: "dezembro"}
aba_mes = int(fechamento_fatura.split('/')[1]) + 1
if aba_mes == 13:
    aba_mes = 1


In [42]:
# Setar o range da aba mês api!C2 - Fechamento da Fatura
range_aba = f'{dicionario_meses[aba_mes]}!F11'

In [43]:
print(f'Fechamento da próxima fatura Visa: {fechamento_fatura}')
pprint(f'Histórico de compras Visa: {lista_definitiva}')

Fechamento da próxima fatura Visa: 23/03/2023
("Histórico de compras Visa: [['10/03', 'ROSANGELA ALMEIDA', 11.0, 0], "
 "['10/03', 'CENTRO EDUCACIONAL ADV', 1126.2, 0], ['10/03', 'RESTAURANTE DA "
 "AN', 30.8, 0], ['10/03', 'AUTO POSTO BRAGA LTDA', 40.0, 0], ['11/03', "
 "'DROGARIA SAMPAIO', 33.5, 0]]")


In [44]:
SERVICE_ACCOUNT_FILE = 'C:\\Users\\S027668971\\Developments\\Google-sheet\\keys.json'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
creds = None
creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)

In [45]:
# The ID spreadsheet.
SAMPLE_SPREADSHEET_ID = '1QhNbaGgEAUzUL_FhOI5xfqTe643xI7lxmpKysub8lPI'
service = build('sheets', 'v4', credentials=creds)
sheet = service.spreadsheets()

In [46]:
# Inserir data de fechamento da fatura visa
sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                    range=range_aba,
                    valueInputOption="USER_ENTERED",
                    body={"values": [[fechamento_fatura]]}).execute()

{'spreadsheetId': '1QhNbaGgEAUzUL_FhOI5xfqTe643xI7lxmpKysub8lPI',
 'updatedRange': 'Abril!F11',
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [ ]:
# Atualizar as planilhas
# descricao_compra = []
# for gasto in lista_definitiva:
#     preço = str(f'{gasto[2]:.2f}').split('.')
#     gasto[2] = "R$ " + preço[0] + "," + preço[1]
#     descricao_compra.append(gasto)
# pprint(descricao_compra)

In [47]:
# Pegar data atual para decidir qual mês irei adicionar os dados
data_atual = date.today() 
ano_atual = data_atual.year
print(ano_atual)

2023


In [62]:
# lista_definitiva2 = [
#     ['28/02', 'Tênis Rafael', 169.0, 2],
#     ['27/02', 'Teste', 29.95, 0],
#     ['07/03', 'Flores', 100.0, 2],
#     ['07/03', 'MIQUEIAS', 62.5, 0]
#     ]

In [54]:
# Alimentar a planilha com os dados do cartão Smiles Visa e Elo Nanquim
from time import mktime

for gasto in lista_definitiva:

    mes_gasto = int(gasto[0].split('/')[1])
    data_gasto = f'{gasto[0]}/{ano_atual}'

    # Transformar data de gasto em timestamp
    timestamp_data_gasto = mktime(datetime.strptime(
                                data_gasto, '%d/%m/%Y').timetuple())

    # Pegar data de fechamento equivalente ao mês do gasto
    range = f'{dicionario_meses[mes_gasto+1]}!F11'
    fechamento_anterior = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                        range=range).execute().get('values', [])[0][0]
    
    # Transformar data de fechamento em timestamp
    timestamp_data_fechamento = mktime(datetime.strptime(
                                fechamento_anterior, '%d/%m/%Y').timetuple())

    # Condição que determina qual mês o gasto será armazenado
    if timestamp_data_gasto < timestamp_data_fechamento:
        # Condição que verifica se o gasto foi parcelado
        if gasto[3]: # Se vier a flag de parcelado
            valor_total = float(gasto[2])
            xparcelas = gasto[3]
            valor_parcela = round((valor_total / xparcelas),2)
            valor_parcela = ("{:.2f}".format(valor_parcela)).replace('.', ',')
            valor_parcela = 'R$ ' + valor_parcela

            # Loop que fará o registro das parcelas de acordo
            count = 1
            while count <= xparcelas:
                parcela = [gasto[0], f'{gasto[1]} {count}/{xparcelas}', valor_parcela]
                # Pegando todos os gastos armazenados e determinando a próxima linha vazia
                range = f'{dicionario_meses[mes_gasto + count]}!H2:J100'
                result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                    range=range).execute().get('values', [])
                next_empty_row = len(result) + 2
                # Condição que compara se gasto se foi armazenado
                if parcela not in result:
                    # Armazenando os gastos na planilha
                    range = f'{dicionario_meses[mes_gasto + count]}!H{next_empty_row}'
                    sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range=range,
                                valueInputOption="USER_ENTERED",
                                body={"values": [parcela]}).execute()
                count += 1

        else: # Se a flag de gasto parcelado vier 0
            valor_gasto = 'R$ ' + ("{:.2f}".format(gasto[2])).replace('.', ',')

            # Condição para corrigir a inserção de valores maiores que um mil
            if len(valor_gasto) > 9:
                valor_gasto = valor_gasto[:4] + '.' + valor_gasto[4:]

            parcela = [gasto[0], gasto[1], valor_gasto]
            # Pegando todos os gastos armazenados e determinando a próxima linha vazia
            range = f'{dicionario_meses[mes_gasto+1]}!H2:J100'
            result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range=range).execute().get('values', [])
            next_empty_row = len(result) + 2
            print(parcela)

            # Condição que compara se gasto se foi armazenado
            if parcela not in result:
                # Armazenando os gastos na planilha
                range = f'{dicionario_meses[mes_gasto+1]}!H{next_empty_row}'
                sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                            range=range,
                            valueInputOption="USER_ENTERED",
                            body={"values": [parcela]}).execute()
                
    else: # Se a compra for feita no mês anterior ao do fechamento da fatura
        # Condição que verifica se o gasto foi parcelado
        if gasto[3]: # Se vier a flag de parcelado
            valor_total = float(gasto[2])
            xparcelas = gasto[3]
            valor_parcela = round((valor_total / xparcelas),2)
            valor_parcela = ("{:.2f}".format(valor_parcela)).replace('.', ',')
            valor_parcela = 'R$ ' + valor_parcela
            print(gasto)

            # Loop que fará o registro das parcelas de acordo
            count = 1
            while count <= xparcelas:
                parcela = [gasto[0], f'{gasto[1]} {count}/{xparcelas}', valor_parcela]
                # Pegando todos os gastos armazenados e determinando a próxima linha vazia
                range = f'{dicionario_meses[mes_gasto + count + 1]}!H2:J100'
                result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                    range=range).execute().get('values', [])
                next_empty_row = len(result) + 2
                # Condição que compara se gasto se foi armazenado
                if parcela not in result:
                    # Armazenando os gastos na planilha
                    range = f'{dicionario_meses[mes_gasto + count + 1]}!H{next_empty_row}'
                    sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range=range,
                                valueInputOption="USER_ENTERED",
                                body={"values": [parcela]}).execute()
    
                count += 1
        else: # Se a flag de gasto parcelado vier 0
            valor_gasto = 'R$ ' + ("{:.2f}".format(gasto[2])).replace('.', ',')

            # Condição para corrigir a inserção de valores maiores que um mil
            if len(valor_gasto) > 9:
                valor_gasto = valor_gasto[:4] + '.' + valor_gasto[4:]
                
            parcela = [gasto[0], gasto[1], valor_gasto]
            # Pegando todos os gastos armazenados e determinando a próxima linha vazia
            range = f'{dicionario_meses[mes_gasto+2]}!H2:J100'
            result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range=range).execute().get('values', [])
            next_empty_row = len(result) + 2
            print(parcela)

            # Condição que compara se gasto se foi armazenado
            if parcela not in result:
                # Armazenando os gastos na planilha
                range = f'{dicionario_meses[mes_gasto+2]}!H{next_empty_row}'
                sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                    range=range,
                                    valueInputOption="USER_ENTERED",
                                    body={"values": [parcela]}).execute()


['10/03', 'ROSANGELA ALMEIDA', 'R$ 11,00']
['10/03', 'CENTRO EDUCACIONAL ADV', 'R$ 1.126,20']
['10/03', 'RESTAURANTE DA AN', 'R$ 30,80']
['10/03', 'AUTO POSTO BRAGA LTDA', 'R$ 40,00']
['11/03', 'DROGARIA SAMPAIO', 'R$ 33,50']


In [52]:
range = f'{dicionario_meses[mes_gasto+1]}!H64:J64'
result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                        range=range).execute().get('values', [])
pprint(result)
valor = 'R$ 1126,20'
if len(valor) > 9:
    valor = valor[:4] + '.' + valor[4:]
    print(valor)

[['10/03', 'CENTRO EDUCACIONAL ADV', 'R$ 1.126,20']]
R$ 1.126,20
